#### 作业六

在基于CIFAR-10数据集的模型(ResNet)训练中结合多种(不少于3种)不同的图像增广方法。它能提高测试准确性吗？试分析其原因。

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# 定义图像增广方法
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomCrop(32, padding=4),  # 随机裁剪
    transforms.RandomRotation(10),  # 随机旋转
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # 色彩抖动
    transforms.ToTensor(),  # 转为张量
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

# 加载CIFAR-10数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# 定义ResNet模型
resnet = models.resnet18(pretrained=False)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# 训练模型
for epoch in range(20):  # 训练20个epoch
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # 每100个batch打印一次损失
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test images: %d %%' % (100 * correct / total))


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


c:\Users\94506\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\94506\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[1,   100] loss: 2.043
[1,   200] loss: 1.725
[1,   300] loss: 1.625
[2,   100] loss: 1.488
[2,   200] loss: 1.451
[2,   300] loss: 1.411
[3,   100] loss: 1.314
[3,   200] loss: 1.284
[3,   300] loss: 1.261
[4,   100] loss: 1.196
[4,   200] loss: 1.178
[4,   300] loss: 1.164
[5,   100] loss: 1.112
[5,   200] loss: 1.077
[5,   300] loss: 1.082
[6,   100] loss: 1.019
[6,   200] loss: 1.047
[6,   300] loss: 1.033
[7,   100] loss: 0.981
[7,   200] loss: 0.984
[7,   300] loss: 0.983
[8,   100] loss: 0.937
[8,   200] loss: 0.936
[8,   300] loss: 0.931
[9,   100] loss: 0.919
[9,   200] loss: 0.905
[9,   300] loss: 0.891
[10,   100] loss: 0.874
[10,   200] loss: 0.868
[10,   300] loss: 0.879
[11,   100] loss: 0.848
[11,   200] loss: 0.852
[11,   300] loss: 0.831
[12,   100] loss: 0.819
[12,   200] loss: 0.809
[12,   300] loss: 0.812
[13,   100] loss: 0.789
[13,   200] loss: 0.797
[13,   300] loss: 0.794
[14,   100] loss: 0.764
[14,   200] loss: 0.779
[14,   300] loss: 0.780
[15,   100] loss: 0